In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

<hr>

In [2]:
data=pd.read_csv("data.csv")

In [3]:
data.head()

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure,Precip
0,2010-01-01 00:51:00,6.0,33.0,32.0,31.0,92.0,0.0,0.0,29.97,29.99,0.01
1,2010-01-01 01:51:00,6.0,33.0,33.0,32.0,96.0,0.0,0.0,29.97,29.99,0.02
2,2010-01-01 02:51:00,5.0,33.0,33.0,32.0,96.0,0.0,0.0,29.97,29.99,0.02
3,2010-01-01 03:51:00,5.0,33.0,33.0,32.0,96.0,0.0,0.0,29.95,29.97,0.02
4,2010-01-01 04:51:00,5.0,33.0,32.0,31.0,92.0,0.0,0.0,29.93,29.96,0.02


<hr>

Pre-Process

In [4]:
import preprocess
preprocessor= preprocess.calculate()

In [5]:
data=preprocessor.remove_columns(data,['DATE','Precip','WETBULBTEMPF','DewPointTempF','StationPressure'])
data=preprocessor.replaceInvalidValueswithNull(data)
is_null_present=preprocessor.is_null_present(data)
if(is_null_present):
    data=preprocessor.impute_missing_values(data)

X,Y=preprocessor.separate_label_feature(data,label_column_name='VISIBILITY')

In [6]:
data.head()

,VISIBILITY,DRYBULBTEMPF,RelativeHumidity,WindSpeed,WindDirection,SeaLevelPressure
0,6.0,33.0,92.0,0.0,0.0,29.99
1,6.0,33.0,96.0,0.0,0.0,29.99
2,5.0,33.0,96.0,0.0,0.0,29.99
3,5.0,33.0,96.0,0.0,0.0,29.97
4,5.0,33.0,92.0,0.0,0.0,29.96


In [7]:
y_test=data["VISIBILITY"]
data=data.drop(["VISIBILITY"],axis=1)

In [8]:
data.head()

,DRYBULBTEMPF,RelativeHumidity,WindSpeed,WindDirection,SeaLevelPressure
0,33.0,92.0,0.0,0.0,29.99
1,33.0,96.0,0.0,0.0,29.99
2,33.0,96.0,0.0,0.0,29.99
3,33.0,96.0,0.0,0.0,29.97
4,33.0,92.0,0.0,0.0,29.96


<hr>

Find Cluster

In [9]:
data_scaled = pd.DataFrame(preprocessor.standardScalingData(data),columns=data.columns)

In [10]:
import file_methods
file_op = file_methods.File_Operation()

In [11]:
kmeans=file_op.load_model('KMeans')

In [12]:
clusters=kmeans.predict(data_scaled)#drops the first column for cluster prediction
data_scaled['clusters']=clusters
clusters=data_scaled['clusters'].unique()
result=[]

<hr>

Predict model by using suitable model

In [13]:
for i in clusters:
    cluster_data = data_scaled[data_scaled['clusters'] == i]
    cluster_data = cluster_data.drop(['clusters'], axis=1)
    model_name = file_op.find_correct_model_file(i)
    model = file_op.load_model(model_name)
    for val in (model.predict(cluster_data.values)):
        result.append(val)
    result = pd.DataFrame(result,columns=['Predictions'])
    # path="Prediction_Output_File/Predictions.csv"
    # result.to_csv("Prediction_Output_File/Predictions.csv",header=True) #appends result to prediction file
    

    Predictions
0      9.574135
1      9.921130
2      9.921130
3     10.086858
4      9.150892
5      9.574135
6      9.756481
7      8.151039
8      7.380515
9      9.478315
10     9.226495
11     9.701770
12     9.993697
13     9.983497
14     9.979990
15     9.992899
16     9.992899
17    10.032632
18     9.986346
19     9.987143
20    10.037550
21    10.019258
22    10.148828
23     9.781425
24    10.169779
25    10.129231
26     9.699118
27     9.735288
28     8.553557


<hr>